In [42]:
#Automatically label
import pandas as pd

# Define the path for the video
video_path = 'tracking_data/Tracking_video12.csv'

# Load the CSV file
df = pd.read_csv(video_path)

# Calculate the difference between the current and previous row for 'Left hip_y'
df['Left hip_y_diff'] = df['Left hip_y'].diff()

# Calculate the new pause threshold
new_pause_threshold = df['Left hip_y_diff'].abs().quantile(0.25)

# Define a function to apply robust labeling
def apply_robust_labeling(changes, pause_threshold, min_consecutive=3):
    robust_labels = []
    current_label = "Unknown"
    consecutive_count = 0

    for change in changes:
        if pd.isna(change):
            # First data point
            new_label = "Unknown"
        elif abs(change) <= pause_threshold:
            new_label = "Pause"
        elif change > 0:
            new_label = "Descending"
        else:
            new_label = "Ascending"

        # Check if the new label is the same as the current label
        if new_label == current_label:
            consecutive_count += 1
        else:
            consecutive_count = 1
            current_label = new_label

        # Update label if there are enough consecutive labels
        if consecutive_count >= min_consecutive:
            robust_labels.append(current_label)
        else:
            robust_labels.append("Transition")

    return robust_labels

# Apply robust labeling to the dataset
df['Label'] = apply_robust_labeling(df['Left hip_y_diff'], new_pause_threshold)

# Drop the 'Left hip_y_diff' column as it's no longer needed
df = df.drop('Left hip_y_diff', axis=1)

# Save the dataframe to a new CSV file
csv_file_name = video_path.split('/')[-1].split('.')[0] + '_labled.csv'
df.to_csv('tracking_data/' + csv_file_name, index=False)


In [49]:
# Plot with lables
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Usage
csv_file_path = (
    "tracking_data/" + video_path.split("/")[-1].split(".")[0] + "_labled.csv"
)
ignore_start = 0  # Number of frames to ignore at the start
ignore_end = 0  # Number of frames to ignore at the end


def plot_movement_data(csv_file_path, ignore_start=0, ignore_end=0):
    # Read the data from the CSV file
    data = pd.read_csv(csv_file_path, delimiter=",")

    # Ignore specified amount of frames at the beginning and/or end
    data = data[ignore_start : -ignore_end or None]

    # Calculate the average hip height
    data["Average_Hip_Height"] = (data["Left hip_y"] + data["Right hip_y"]) / 2

    # Define custom color mapping for movement types
    custom_color_mapping = {
        "Pause": "orange",
        "Ascending": "green",
        "Descending": "red",
        "Transition": "yellow",
        "Unknown": "grey",
    }

    # Create a scatter plot
    fig = go.Figure()

    for label, color in custom_color_mapping.items():
        mask = data["Label"] == label
        fig.add_trace(
            go.Scatter(
                x=data.loc[mask, "Timestamp"],
                y=data.loc[mask, "Average_Hip_Height"],
                mode="markers",
                name=label,
                marker_color=color,
                hovertemplate="x: %{x}",
            )
        )

    # Get the x values of the first and last data points
    first_x = data["Timestamp"].iloc[0]
    last_x = data["Timestamp"].iloc[-1]

    # Add vertical lines at the first and last data points
    fig.add_shape(
        type="line",
        x0=first_x,
        y0=0,
        x1=first_x,
        y1=1,
        yref="paper",
        line=dict(color="Green", width=2),
    )
    fig.add_shape(
        type="line",
        x0=last_x,
        y0=0,
        x1=last_x,
        y1=1,
        yref="paper",
        line=dict(color="Red", width=2),
    )

    # Add annotations for the first and last data points
    fig.add_annotation(x=first_x, y=0.05, text=f"Start: {first_x}", showarrow=False, yref="paper")
    fig.add_annotation(x=last_x, y=0.05, text=f"End: {last_x}", showarrow=False, yref="paper")

    # Set hovermode to 'x' to display a vertical line and x-value at the cursor's current position
    fig.update_layout(hovermode="x")

    # Increase the number of grid lines
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')


    # Remove the light blue background
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )

    fig.show()


plot_movement_data(csv_file_path, ignore_start, ignore_end)

In [46]:
def remove_ignored_frames(csv_file_path, start_frame=0, end_frame=None):
    # Read the data from the CSV file
    data = pd.read_csv(csv_file_path, delimiter=",")

    # Ignore specified amount of frames at the beginning and/or end
    data = data[start_frame:end_frame]

    # Save the modified data back to the CSV file
    data.to_csv(csv_file_path, index=False)

# Usage
start_frame = 81  # First frame to keep
end_frame = 1000  # First frame to discard

remove_ignored_frames(csv_file_path, start_frame, end_frame)